loading files

In [44]:
import pandas
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm
# from time import sleep

# for i in trange(3, desc='1st loop'):
#     for j in tqdm(range(100), desc='2nd loop'):
#         sleep(0.01)

In [45]:
path_to_snp_statistics = 'input/SNP_statistics_file.txt'
path_to_snp_call_contrast_positions = 'input/SNP_call_contrast_positions_file.txt'
path_to_cpf_csv = 'input/cpf.csv'

In [48]:
snp_statistics_df = pandas.read_csv(path_to_snp_statistics, delimiter='\t', header=0)
snp_call_contrast_positions_df = pandas.read_csv(path_to_snp_call_contrast_positions, delimiter='\t', skiprows=5, header=0)

In [51]:
for column_name in trange(snp_call_contrast_positions_df.columns, desc='Removing columns:'):
    if column_name == 'probeset_id':
        pass
    else:
        name_list = column_name.split('.')
        if name_list[-1] == 'CEL_log_ratio':
            #print('preimenuj')
            #print(name_list[0])
            snp_call_contrast_positions_df.rename(columns={column_name: name_list[0]}, inplace=True)
        else:
            #print('ukloni')
            snp_call_contrast_positions_df.drop(column_name, axis=1, inplace=True)

TypeError: 'Index' object cannot be interpreted as an integer

In [15]:
demo_df = pandas.merge(snp_call_contrast_positions_df, snp_statistics_df, on=['probeset_id'], how='inner')


In [16]:
for index, row in demo_df.iterrows():
    if row['n_NC'] >= 20:
        demo_df.drop(index, inplace=True)
demo_df.reset_index(drop=True, inplace=True)

In [17]:
aa_meanx_min_value = demo_df['AA.meanX'].min()
bb_meanx_min_value = demo_df['BB.meanX'].min()
aa_meanx_max_value = demo_df['AA.meanX'].max()
bb_meanx_max_value = demo_df['BB.meanX'].max()
aa_meanx_delta = abs(aa_meanx_max_value) - aa_meanx_min_value
bb_meanx_delta = abs(bb_meanx_max_value) - bb_meanx_min_value

aa_bb = (aa_meanx_delta + bb_meanx_delta) / 2

In [18]:
print(aa_meanx_max_value)
print(aa_meanx_min_value)
print(aa_meanx_delta)


print(bb_meanx_max_value)
print(bb_meanx_min_value)
print(bb_meanx_delta)

print(aa_bb)


4.859
-1.171
6.03
1.603
-4.41
6.013
6.0215


In [19]:
for index, row in demo_df.iterrows():
    demo_df.at[index, 'AA-BB'] = demo_df.at[index, 'AA.meanX'] - demo_df.at[index, 'BB.meanX']

In [38]:
demo_filtered_df = pandas.DataFrame(columns=demo_df.columns)
for index, row in demo_df.iterrows():
    if row['AA-BB'] > aa_bb:
        demo_filtered_df.loc[len(demo_filtered_df)] = row

In [39]:
columns_to_remove = ['n_NC', 'AA.meanX', 'AA.varX', 'AB.meanX', 'BB.meanX', 'BB.varX', 'AA-BB', 'probeset_id']
demo_filtered_df.drop(columns=columns_to_remove, inplace=True)

In [42]:
output_directory = 'histograms'
import os
os.makedirs(output_directory, exist_ok=True)
for column in demo_filtered_df.columns:
    plt.hist(demo_filtered_df[column], bins=160, edgecolor='black')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.title(f'Histogram for {column}')
    output_file_path = os.path.join(output_directory, f'{column}_histogram.png')
    plt.savefig(output_file_path, bbox_inches='tight')
    #plt.show()
    plt.clf()
plt.close('all')